In [1]:
import torch
import data_utils
from models.DeepAnT import DeepAnT_CNN, DeepAnT_LSTM
from training.DeepAnT_train import DeepAntTrainingPipeline

In [2]:
# Try to use GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# Hyperparameters

In [3]:
# choose between CNN and LSTM absed DeepAnT models
model_type = "DeepAnT_CNN"

In [4]:
if model_type == "DeepAnT_LSTM":
    # Data
    num_features = 7
    seq_length = 30
    seq_stride = 10
    gen_seq_len = seq_length
    # Model
    hidden_dim = 128
    layers = 4
    anm_det_thr = 0.5
    # Training
    num_epochs = 100
    batch_size = 8
    lr=1e-5
    wd=5e-6
elif model_type == "DeepAnT_CNN":
    # Data
    num_features = 7
    seq_length = 30
    seq_stride = 10
    gen_seq_len = 1
    # Model
    anm_det_thr = 0.8
    # Training
    num_epochs = 100
    batch_size = 8
    lr=1e-5
    wd=5e-6

# Load data

In [5]:
#train_dl, test_dl = data_utils.kdd99(seq_length, seq_stride, num_features, gen_seq_len, batch_size)

In [6]:
file_path = './data/Stocks/aapl.us.txt'
tscv_dl_list = data_utils.load_stock_as_crossvalidated_timeseries(file_path, seq_length, seq_stride, gen_seq_len, batch_size)

# Model

In [7]:
if model_type == "DeepAnT_LSTM":
    model = DeepAnT_LSTM(num_features,hidden_dim,layers,anm_det_thr).to(DEVICE)
elif model_type == "DeepAnT_CNN":
    model = DeepAnT_CNN(seq_length,num_features,anm_det_thr).to(DEVICE)

In [8]:
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(list(model.parameters()), lr=lr, weight_decay=wd)

# Training

In [9]:
pipeline = DeepAntTrainingPipeline()

In [10]:
pipeline.train(tscv_dl_list, model, optimizer, criterion, num_epochs, DEVICE)

KeyboardInterrupt: 